<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/05_combined_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-27 02:01:32--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12110 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.83K  --.-KB/s    in 0s      

2022-06-27 02:01:32 (84.9 MB/s) - ‘utils.py’ saved [12110/12110]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.2 MB/s 


In [4]:
BATCH_SIZE = 16
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
target_shape = (32, 48, 1)

rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL, trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL, trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL, trainable=True)


rgb_features = rgb_hub_module(rgb_input)
rgb_layer = keras.layers.Reshape(target_shape=target_shape)(rgb_features)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)
xyz_layer = keras.layers.Reshape(target_shape=target_shape)(xyz_features)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hsv_hub_module(hsv_input)
hsv_layer = keras.layers.Reshape(target_shape=target_shape)(hsv_features)

block = keras.layers.Concatenate()([rgb_layer, xyz_layer, hsv_layer])

conv1 = keras.layers.Conv2D(filters=10,
                            kernel_size=(2,2),
                            strides=(2,3),
                            padding='same',
                            activation='relu')(block)

conv2 = keras.layers.Conv2D(filters=20,
                            kernel_size=(2,2),
                            strides=(2,2),
                            activation='relu')(conv1)

avg_pool = keras.layers.AveragePooling2D(pool_size=(2,2))(conv2)

conv3 = keras.layers.Conv2D(filters=20,
                            kernel_size=(1,1),
                            activation='relu')(avg_pool)

flatten = keras.layers.Flatten()(conv3)

output = keras.layers.Dense(N_CLASSES, activation='softmax')(flatten)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [10]:
model._name = 'combined-model-v1'

In [11]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [13]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy', 
                                              verbose=1,
                                              patience=8,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name)
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [14]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
1037/1037 [==============================] - ETA: 0s - loss: 19.3803 - auc: 0.6741 - accuracy: 0.6029

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


1037/1037 [==============================] - 2472s 2s/step - loss: 19.3803 - auc: 0.6741 - accuracy: 0.6029 - val_loss: 1.1515 - val_auc: 0.7181 - val_accuracy: 0.6255
Epoch 2/15
1037/1037 [==============================] - ETA: 0s - loss: 12.5516 - auc: 0.8228 - accuracy: 0.7368

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


...

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


1037/1037 [==============================] - 780s 752ms/step - loss: 3.1116 - auc: 0.9755 - accuracy: 0.9239 - val_loss: 0.8803 - val_auc: 0.8544 - val_accuracy: 0.8046
Epoch 8/15
1037/1037 [==============================] - 727s 700ms/step - loss: 2.0482 - auc: 0.9853 - accuracy: 0.9466 - val_loss: 0.9873 - val_auc: 0.8385 - val_accuracy: 0.7989
Epoch 9/15
1037/1037 [==============================] - 728s 701ms/step - loss: 1.5154 - auc: 0.9911 - accuracy: 0.9623 - val_loss: 1.0589 - val_auc: 0.8247 - val_accuracy: 0.7894
Epoch 10/15
1037/1037 [==============================] - 728s 702ms/step - loss: 1.0439 - auc: 0.9947 - accuracy: 0.9722 - val_loss: 1.0081 - val_auc: 0.8420 - val_accuracy: 0.8034
Epoch 11/15
1037/1037 [==============================] - 728s 701ms/step - loss: 0.7486 - auc: 0.9961 - accuracy: 0.9793 - val_loss: 1.0416 - val_auc: 0.8359 - val_accuracy: 0.8046
Epoch 12/15
1037/1037 [==============================] - 729s 702ms/step - loss: 0.5971 - auc: 0.9972 - accur

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


1037/1037 [==============================] - 782s 753ms/step - loss: 0.3290 - auc: 0.9981 - accuracy: 0.9905 - val_loss: 1.1227 - val_auc: 0.8423 - val_accuracy: 0.8240
Epoch 15/15
1037/1037 [==============================] - 727s 701ms/step - loss: 0.2894 - auc: 0.9992 - accuracy: 0.9916 - val_loss: 1.2316 - val_auc: 0.8326 - val_accuracy: 0.8160


In [15]:
!cp  -r /content/logs/checkpoints/combined-model-v1 /content/drive/MyDrive/checkpoints/combined-v1